# Model Discovery in Declare4Py

This tutorial explains how to perform the discovery of a DECLARE model and how to browse the results.

After importing the Declare4Py package, a `Declare4Py` object has to be instantiated to load the log.

In [1]:

from declare4py.declare4py import Declare4Py


log_path = r"C:\Users\peete\OneDrive\Documenten\School\2e master BI\Masterproef\Coding\Logs for testing\Level1.xes"

d4py = Declare4Py()
d4py.parse_xes_log(log_path)

c:\Users\peete\AppData\Local\Programs\Python\Python310\lib\site-packages\pm4py\util\dt_parsing\parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(
c:\Users\peete\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 5567.17it/s]


The discovery of a DECLARE model is performed with two consecutive steps in Declare4Py:

1. the computing of the frequent itemsets with an input support and of length 2 with the `compute_frequent_itemsets()` function, see the tutorial on the analysis of the logs.
2. the model discovery with the `discovery()` function. This takes as input the boolean parameter `consider_vacuity=true` that considers vacuously satisfied traces as satisfied, violated otherwise. The integer parameter `max_declare_cardinality` sets the cardinality of the Exactly, Existence and Absence templates. This function returns a Python dictionary containing the results indexed by discovered constraints. The value is a Python dictionary with keys the single ids of the traces in the log (that can be retrieved with `get_trace_keys()`) that satisfy the constraint. The values of this inner dictionary is a `CheckerResult` object containing the number of pendings, activations, violations, fulfilments and the truth value of the trace for that constraint.
```
discovery_results = {constr_1: {trace_1: CheckerResult object, trace_2: CheckerResult object, ...},
                     constr_2: {trace_1: CheckerResult object, ... },
                     ...
                    }
```
The `CheckerResult` objects can be accessed by the attributes `num_pendings`, `num_activations`, `num_fulfillments`, `num_violations` and `state`.

In [7]:
d4py.compute_frequent_itemsets(min_support=0.9, len_itemset=6)
discovery_results = d4py.discovery(consider_vacuity=True, max_declare_cardinality=2)

Computing discovery ...


In [8]:
print(discovery_results)

{'Existence1[t] | |': {(0, 'concept:name'): <declare4py.models.checker_result.CheckerResult object at 0x000001F4DAED7370>, (1, 'lifecycle:transition'): <declare4py.models.checker_result.CheckerResult object at 0x000001F4DAED7280>, (2, 'time:timestamp'): <declare4py.models.checker_result.CheckerResult object at 0x000001F4DAED7610>, (3, 'case:concept:name'): <declare4py.models.checker_result.CheckerResult object at 0x000001F4DAED73A0>}, 'Existence2[t] | |': {(1, 'lifecycle:transition'): <declare4py.models.checker_result.CheckerResult object at 0x000001F4DAED7E80>, (2, 'time:timestamp'): <declare4py.models.checker_result.CheckerResult object at 0x000001F4DAED74F0>}, 'Absence2[t] | |': {(0, 'concept:name'): <declare4py.models.checker_result.CheckerResult object at 0x000001F4DAED7580>, (3, 'case:concept:name'): <declare4py.models.checker_result.CheckerResult object at 0x000001F4DAED7640>}, 'Exactly1[t] | |': {(0, 'concept:name'): <declare4py.models.checker_result.CheckerResult object at 0x0

Let's inspect the results for the constraint `Responded Existence[ER Sepsis Triage, ER Triage] | | |` and the trace `(488, 'VR')`

In [3]:
decl_constr = 'Responded Existence[ER Sepsis Triage, ER Triage] | | |'
trace_id = (488, 'VR')
print(f"Number of pendings: {discovery_results[decl_constr][trace_id].num_pendings}")
print(f"Number of activations: {discovery_results[decl_constr][trace_id].num_activations}")
print(f"Number of fulfilments: {discovery_results[decl_constr][trace_id].num_fulfillments}")
print(f"Number of violation: {discovery_results[decl_constr][trace_id].num_violations}")
print(f"Truth value of: {discovery_results[decl_constr][trace_id].state}")

KeyError: 'Responded Existence[ER Sepsis Triage, ER Triage] | | |'

The results of the discovery can be filtered according to a support threshold and saved in a DECLARE file (if specified). This is performed by the `filter_discovery` function that returns a Python dictionary with the discovered DECLARE constraints as keys and their support as value.

In [ ]:
d4py.filter_discovery(min_support=0.7, output_path='sepsis_model_discovered.decl')